In [118]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

from calvin_utils.generate_nifti import view_and_save_nifti

In [119]:
analysis = 'response_topology/voxelwise_mixed_effects/age_mediated_by_grey_matter'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/grey_matter_damage_score_and_outcomes.csv'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/grey_matter_damage_score_and_outcomes.csv'
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    #out_dir = r'path to out dir here'
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    
    print('I have set pathnames in the Mac style')
    print('I will save to :', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\AD_to_memory_net'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')
    
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

I have set pathnames in the Mac style
I will save to : /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter


# Import Clinical Data

In [86]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clin_path)
    
# #Remove subjects
# outlier_index=[11, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)
# data_df.reset_index(drop=True, inplace=True)

data_df

,"Patient # CDR, ADAS",Age,% Change from baseline (ADAS-Cog11),Subiculum Connectivity,Subiculum Damage Score,Hippocampus Damage Score,Temporal Damage Score,Frontal Damage Score,Parietal Damage Score,Cerebellum Damage Score,Insula Damage Score,Occipital Damage Score
0,101,62,-21.428571,56.864683,-22.341183,-19032.589510,-6811.832831,-3390.513012,-6514.359274,-1733.125937,-133.189922,-4146.074230
1,102,77,-36.363636,52.970984,-40.309051,-184720.025100,-12864.565970,-6136.065565,-4548.582996,-5422.573735,-622.591915,-3965.251674
2,103,76,-78.947368,62.459631,-19.785287,-45387.958520,-6591.622671,-8075.228864,-4447.485410,-3607.025644,-624.924211,-2512.288860
3,104,65,-129.411765,59.611631,-42.332132,-60471.819480,-11699.796100,-6244.075797,-5212.034938,-3227.497127,-570.463181,-2934.581194
4,105,50,-10.526316,57.928350,-2.136441,-5968.011901,-4662.284230,-6429.216510,-4540.145985,-1893.423398,-299.251771,-2070.284124
5,106,66,-38.461538,63.323903,-26.237501,-29471.040290,-2479.701861,-2121.181555,-2841.442623,-1631.143552,-120.128438,-2222.047928
6,107,64,-15.384615,53.317851,-43.667344,-118201.273400,-11646.437390,-7046.077001,-8392.540599,-2361.515431,-614.793778,-6163.741802
7,108,60,-29.166667,57.979468,-62.535677,-66947.522780,-10926.588590,-4526.665072,-5782.261434,-3820.805562,-263.752339,-3537.170972
8,109,72,-30.434783,66.950749,-22.137900,-75270.490460,-11252.394020,-5240.470247,-4707.095249,-1171.115413,-261.459686,-3628.614588
9,110,72,-84.615385,69.267271,-6.545406,-40392.854920,-7111.854909,-9629.498968,-5124.972435,-1647.853988,-1035.648091,-2358.679568


In [87]:
subject_column = 'Patient # CDR, ADAS'
outcome_column = '% Change from baseline (ADAS-Cog11)'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
outcomes_df = pd.DataFrame()
outcomes_df['outcome'] = data_df.loc[:, [outcome_column]]

#Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
outcomes_df.iloc[:,:] = scaler.fit_transform(outcomes_df.iloc[:,:])


outcomes_df['subject_id'] = data_df.loc[:, [subject_column]]
# outcomes_df['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] #<------------------------problem code. must generalize to extract subject id better
outcomes_df.set_index('subject_id', inplace=True)

# data_df = data_df.set_index('Patient # CDR, ADAS')
# data_df['subject_id'] = data_df.index
# data_df['outcome'] = data_df

# Convert the 'subject_id' column to strings for each DataFrame
outcomes_df.index = outcomes_df.index.astype(str)
display(outcomes_df)

,outcome
subject_id,
101,0.292279
102,-0.016545
103,-0.897080
104,-1.940569
105,0.517713
106,-0.059924
107,0.417254
108,0.132272
109,0.106051


Get More Clinical Data As Needed

In [88]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = 'Age'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1['clinical_data'] = data_df.loc[:, [clinical_information_column]]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
display(clinical_df_1)

,clinical_data
subject_id,
101,62
102,77
103,76
104,65
105,50
106,66
107,64
108,60
109,72


Import Imaging Data

In [89]:
from calvin_utils.import_matrices import import_matrices_from_folder
from calvin_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='/*_T*.nii*')

#Standardize matrix via z score
prepped_matrix = pd.DataFrame()
from calvin_utils.z_score_matrix import z_score_matrix
for column in matrix.columns:
    #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
    print(column)
    new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA
    #Standardize column by column
    prepped_matrix[new_name] = z_score_matrix(matrix[column])
    #Set values outside brain back to zero
    prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
prepped_matrix = prepped_matrix.transpose()
prepped_matrix['subject_id'] = [str(col).split('_')[0] for col in prepped_matrix.index]
prepped_matrix.set_index('subject_id', inplace=True)
neuroimaging_df_1 = prepped_matrix
neuroimaging_df_1.index = neuroimaging_df_1.index.astype(str)
#Display results
display(neuroimaging_df_1)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*_T*.nii*
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further

,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Import More Imaging Data

In [90]:
#Import Another Dataframe if desired
from calvin_utils.import_matrices import import_matrices_from_folder
from calvin_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='/*/*/*/*mwp1*resamp*.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------
#Standardize matrix via z score
# neuroimaging_df_2 = pd.DataFrame()
# from calvin_utils.z_score_matrix import z_score_matrix
# if performed_z_score:
#     for column in matrix.columns:
#         #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
#         new_name = int(column.split('_')[0])
#         #Standardize column by column
#         prepped_matrix[new_name] = z_score_matrix(matrix[column])
#         #Set values outside brain back to zero
#         prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
display(neuroimaging_df_2)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170/*/*/*/*mwp1*resamp*.nii*


,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_df = data_df.loc[:, ['Age', '% Change from baseline (ADAS-Cog11)']]

In [ ]:
data_df

## Optional - Join Imaging Data to the Clinical Data

In [ ]:
#only operate within the brain mask
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]
prepped_matrix = prepped_matrix.iloc[brain_indices, :].reset_index()
prepped_matrix = prepped_matrix.transpose()

#Now, join voxels to the clinical data
regression_df = data_df.merge(neuroimaging_df_1, on='Patient # CDR, ADAS', right_index=False)

display(regression_df)

# perform masking

In [106]:
#Mask Dataframes
mask_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)

In [92]:
neuroimaging_df_2 = neuroimaging_df_2.reindex(neuroimaging_df_1.index).dropna()
neuroimaging_df_2

,102956,102957,102958,102959,102960,103046,103047,103048,103049,103050,...,797919,798009,798010,798099,798100,798190,798191,798281,798282,798373
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.229362,0.205727,0.053395,0.074691,0.113833,0.423250,0.513379,0.377440,0.153252,0.007786,...,0.463366,0.303836,0.372540,0.605460,0.464765,0.746839,0.739713,0.870503,0.752853,0.769740
139,0.109401,0.150143,0.197897,0.425160,0.362449,0.138988,0.176965,0.240504,0.407940,0.543465,...,0.066427,0.405873,0.192811,0.508609,0.499546,0.555987,0.478337,0.583788,0.478562,0.429630
147,0.162510,0.179246,0.263863,0.298417,0.158954,0.351168,0.314347,0.409709,0.518645,0.382331,...,0.110859,0.403632,0.583603,0.690095,0.910620,0.837578,0.951557,0.822104,0.886667,0.739561
135,0.153985,0.113712,0.141172,0.173383,0.114053,0.193699,0.248250,0.240887,0.267419,0.182920,...,0.068370,0.154941,0.097959,0.284367,0.265228,0.388628,0.386707,0.442965,0.473160,0.499106
101,0.364223,0.145639,0.089289,0.140574,0.196354,0.665556,0.729897,0.594886,0.243001,0.039168,...,0.227021,0.293693,0.213777,0.688021,0.515107,0.662774,0.660089,0.682468,0.690582,0.717662
150,0.246922,0.405237,0.310447,0.300769,0.206188,0.243030,0.413811,0.660632,0.502521,0.380426,...,0.058324,0.354329,0.274899,0.790549,0.449493,0.734528,0.539761,0.682080,0.574928,0.593957
122,0.204949,0.197069,0.196635,0.160092,0.044966,0.266371,0.292215,0.281765,0.244403,0.121008,...,0.118555,0.191085,0.216092,0.368782,0.331164,0.385139,0.412472,0.474175,0.450113,0.474031
146,0.175004,0.321799,0.394048,0.364054,0.238638,0.406729,0.353518,0.489140,0.512552,0.508472,...,0.404712,0.447947,0.505782,0.575915,0.614407,0.668394,0.675969,0.663611,0.682129,0.510989
138,0.188685,0.139497,0.121198,0.156062,0.143464,0.292154,0.373723,0.256809,0.156228,0.161139,...,0.211858,0.634101,0.550631,0.687401,0.573499,0.555129,0.571267,0.448494,0.551988,0.422085


## Generate Statistical Maps

Moderated Mediation Analysis

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df

def perform_mediated_moderation_analysis(dataframe, exposure, mediator, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a mediated moderation analysis by estimating the joint indirect effects of an exposure variable
    through a mediator on a dependent variable using bootstrapping, considering the moderating effect of another variable.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable.
    - mediator: str, column name of the mediator variable.
    - moderator: str, column name of the moderator variable.
    - dependent_variable: str, column name of the dependent variable.
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for the indirect effect.

    Example Usage:
    result_df = perform_mediated_moderation_analysis(data_df, exposure='Age',
                                                     mediator='Brain_Lobe',
                                                     moderator='Stimulation',
                                                     dependent_variable='Outcome')
    """

    ab_paths = []

    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)

        # Fit the models and calculate the indirect effect for this bootstrap sample
        model_M = ols(f"{mediator} ~ {moderator}", data=sample).fit()
        model_Y = ols(f"{dependent_variable} ~ {exposure} + {mediator} + {moderator} + {exposure}:{mediator} + {exposure}:{moderator}", data=sample).fit()

        indirect_effect = model_M.params[moderator] * model_Y.params[f'{exposure}:{mediator}']

        # Append the indirect effect to the list
        ab_paths.append(indirect_effect)
    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, mediators=mediator)

    return result_df


def voxelwise_mediated_moderation_analysis(mediator_df, moderator_df, exposure_df, outcome_df, bootstrap_samples=5000):
    """
    Perform voxelwise mediated moderation analysis between the corresponding voxels from
    neuroimaging dataframes and clinical dataframes on a patient's outcome.
    
    Parameters:
        mediator_df (pd.DataFrame): DataFrame containing the mediator variable with patients in rows and 'subject_id' as index.
        moderator_df (pd.DataFrame): DataFrame containing the moderator variable with patients in rows and 'subject_id' as index.
        exposure_df (pd.DataFrame): DataFrame containing the exposure variable with patients in rows and 'subject_id' as index.
        outcome_df (pd.DataFrame): DataFrame containing the outcome variable in 'outcome' column with patients in rows and 'subject_id' as index.
        bootstrap_samples (int): Number of bootstrap samples to use in mediated moderation analysis.
    
    Returns:
        results_df (pd.DataFrame): DataFrame containing mediated moderation analysis results for each voxel.
    """
    
    # Number of voxels
    n_voxels = exposure_df.shape[1]

    # Initialize a list to store the results for each voxel
    results = []

    # Loop through each voxel and perform mediated moderation analysis
    for i in tqdm(range(n_voxels)):
        # Create temporary dataframe with outcome, mediator, moderator, and exposure data for the current voxel
        temp_df = pd.DataFrame({
            'exposure': exposure_df.iloc[:, i] if exposure_df.shape[1] > 1 else exposure_df.iloc[:, 0],
            'mediator': mediator_df.iloc[:, i] if mediator_df.shape[1] > 1 else mediator_df.iloc[:, 0],
            'moderator': moderator_df.iloc[:, i] if moderator_df.shape[1] > 1 else moderator_df.iloc[:, 0],
            'outcome': outcome_df['outcome']
        })

        # Perform mediated moderation analysis on the temporary dataframe
        result_df = perform_mediated_moderation_analysis(dataframe = temp_df,
                                                         exposure = 'exposure', 
                                                         mediator = 'mediator', 
                                                         moderator = 'moderator', 
                                                         dependent_variable ='outcome', 
                                                         bootstrap_samples=bootstrap_samples)
        # Append voxel index to result_df
        result_df['voxel_index'] = i
        results.append(result_df)

    # Concatenate all result dataframes into one
    results_df = pd.concat(results, ignore_index=True)

    return results_df


In [ ]:
results_df = voxelwise_mediated_moderation_analysis(mediator_df=neuroimaging_df_2,
                                                    moderator_df=clinical_df_1, 
                                                    exposure_df=neuroimaging_df_1, 
                                                    outcome_df=outcomes_df, 
                                                    bootstrap_samples=2)

Perform Multiprocssed Voxelwise Mediated Moderation

In [101]:
import multiprocessing as mp

import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df

def perform_mediated_moderation_analysis(dataframe, exposure, mediator, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a mediated moderation analysis by estimating the joint indirect effects of an exposure variable
    through a mediator on a dependent variable using bootstrapping, considering the moderating effect of another variable.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable.
    - mediator: str, column name of the mediator variable.
    - moderator: str, column name of the moderator variable.
    - dependent_variable: str, column name of the dependent variable.
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for the indirect effect.

    Example Usage:
    result_df = perform_mediated_moderation_analysis(data_df, exposure='Age',
                                                     mediator='Brain_Lobe',
                                                     moderator='Stimulation',
                                                     dependent_variable='Outcome')
    """

    ab_paths = []

    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)

        # Fit the models and calculate the indirect effect for this bootstrap sample
        model_M = ols(f"{mediator} ~ {moderator}", data=sample).fit()
        model_Y = ols(f"{dependent_variable} ~ {exposure} + {mediator} + {moderator} + {exposure}:{mediator} + {exposure}:{moderator}", data=sample).fit()

        indirect_effect = model_M.params[moderator] * model_Y.params[f'{exposure}:{mediator}']
        ab_paths.append(indirect_effect)
    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, mediators=mediator)

    return result_df

def process_voxel(i, exposure_df, mediator_df, moderator_df, outcome_df, bootstrap_samples):
    # This function will be applied to each voxel

    # Compose the voxelwise data dictionary
    temp_dict = {
        'exposure': exposure_df.iloc[:, i] if exposure_df.shape[1] > 1 else exposure_df.iloc[:, 0],
        'mediator': mediator_df.iloc[:, i] if mediator_df.shape[1] > 1 else mediator_df.iloc[:, 0],
        'moderator': moderator_df.iloc[:, i] if moderator_df.shape[1] > 1 else moderator_df.iloc[:, 0],
        'outcome': outcome_df['outcome']
    }
    
    # Perform the mediated moderation analysis for the voxel
    voxel_result = perform_mediated_moderation_analysis(dataframe = pd.DataFrame(temp_dict),
                                                         exposure = 'exposure', 
                                                         mediator = 'mediator', 
                                                         moderator = 'moderator', 
                                                         dependent_variable ='outcome', 
                                                         bootstrap_samples=10)

    return voxel_result


In [103]:
from joblib import Parallel, delayed

def voxelwise_mediated_moderation_analysis(exposure_df, mediator_df, moderator_df, outcome_df, bootstrap_samples):
    n_voxels = exposure_df.shape[1]

    # Define the iterable inputs
    iterable = range(n_voxels)

    # Initialize a parallel joblib instance with verbose=10 to get progress updates
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_voxel)(i, exposure_df, mediator_df, moderator_df, outcome_df, bootstrap_samples) for i in iterable
    )

    # Combine the results and return
    combined_df = pd.concat([result.reset_index(drop=True) for result in results], ignore_index=True)
    return combined_df

# Run the voxel-wise analysis
results_df = voxelwise_mediated_moderation_analysis(mediator_df=neuroimaging_df_2.iloc[:,1:10],
                                                    moderator_df=clinical_df_1, 
                                                    exposure_df=neuroimaging_df_1.iloc[:,1:10], 
                                                    outcome_df=outcomes_df, 
                                                    bootstrap_samples=2)
results_df

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator       -0.015589         -0.041651           0.005475      0.2
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator        0.006017         -0.012571           0.028055      0.4
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator       -0.006671         -0.064615           0.045726      0.4
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator        0.013123         -0.024497           0.055083      0.4
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator        0.004044         -0.012499           0.018868      0.1
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator        0.015002         -0.000784           0.042877      0.1
          Point Estimate  2.5th Percentile  97.5th Percentile  P-value
mediator        0.010937          -0.01924           0.042054      0.3
      

[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    1.7s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    1.7s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    1.7s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:    1.7s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    1.7s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    1.7s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.8s finished


,Point Estimate,2.5th Percentile,97.5th Percentile,P-value
0,-0.015589,-0.041651,0.005475,0.2
1,0.006017,-0.012571,0.028055,0.4
2,-0.006671,-0.064615,0.045726,0.4
3,0.013123,-0.024497,0.055083,0.4
4,0.004044,-0.012499,0.018868,0.1
5,0.015002,-0.000784,0.042877,0.1
6,0.010937,-0.019240,0.042054,0.3
7,0.017839,-0.007352,0.039854,0.1
8,0.002419,-0.021010,0.024960,0.5


In [115]:
# mask_path = None #'/Users/cu135/Dropbox (Partners HealthCare)/resources/mni_spaces/6th_gen/mni_152_gm_mask_resampled.nii'
# masking_df = None

results_df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/mediated_moderation/age_mediated_by_grey_matter/P-value_mediated_moderation_analysis_results.csv')

mask_threshold=0
#----------------------------------------------------------------DO NOT TOUCH
#Prepare Indices to Unmask
from calvin_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
from calvin_utils.generate_nifti import view_and_save_nifti
from nilearn import image

#Perform Unmasking
if mask_path is not None:
    mni_mask = image.load_img(mask_path).get_fdata().flatten()
    brain_indices = np.where(mni_mask > mask_threshold)[0]
elif masking_df is not None:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    mask = masking_df.transpose().reset_index(drop=True).copy()
    mask['mask_index'] = mask.sum(axis=1)
    brain_indices = np.where(mask['mask_index'] != 0)[0]
else:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0)[0]

# Create a boolean mask for brain_indices
bool_mask = np.zeros_like(mni_mask, dtype=bool)
bool_mask[brain_indices] = True

for statistic in results_df.columns:
    try:
        print(statistic)
        
        # Initialize the output mask with NaN values
        output_mask = np.full_like(mni_mask, np.nan)
        
        # Reinstate the values at brain_indices
        output_mask[bool_mask] = results_df[statistic]
        
        # View and save
        view_and_save_nifti(output_mask, out_dir=out_dir, output_name=statistic)
    except:
        print(f"Couldn't convert {statistic} to nifti")

Unnamed: 0
(91, 109, 91)
(902629,)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter
Point Estimate
(91, 109, 91)
(902629,)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter
2.5th Percentile
(91, 109, 91)
(902629,)
Image saved to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter
97.5th Percentile
(91, 109, 91)
(902629,)
Image saved to: 
 /Users/cu135/Dropb

Voxelwise Regression Using Multiple Neuroimaging Files

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm
import numpy as np


def voxelwise_multiple_neuroimages_regression(outcome_df, neuroimaging_df, another_voxelwise_df, model_type='linear', include_interaction=False):
    """
    Perform voxelwise regression with interactions between the corresponding voxels from 
    neuroimaging_df and another_voxelwise_df on a patient's outcome.
    
    Parameters:
        outcome_df (pd.DataFrame): DataFrame containing the outcome variable in 'outcome' column with patients in rows and 'subject_id' as index. 
        neuroimaging_df (pd.DataFrame): DataFrame containing voxelwise neuroimaging data with patients in rows and 'subject_id' as index.
        another_voxelwise_df (pd.DataFrame): DataFrame containing another set of voxelwise data with patients in rows and 'subject_id' as index.
        model_type (str): Specifies the type of regression model to use ('linear' or 'logistic').
        include_interaction (bool): If True, includes the interaction between the voxels in the regression model.
    
    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values, coefficient values, t-values, 
                                   F-statistics, p-values of the F-statistic, and R-squared values for each voxel.
    """
    
    # Ensure that the number of voxels is the same in both neuroimaging_df and another_voxelwise_df
    assert neuroimaging_df.shape[1] == another_voxelwise_df.shape[1], "Mismatch in number of voxels"
    
    # Number of voxels
    n_voxels = neuroimaging_df.shape[1]

    # Initialize a list to store the results for each voxel
    results = []

    # Loop through each voxel and perform regression
    for i in tqdm(range(0, n_voxels)):
        # Create temporary dataframe with outcome and corresponding voxel from both dataframes
        temp_df = outcome_df[['outcome']].merge(
            neuroimaging_df[[neuroimaging_df.columns[i]]], left_index=True, right_index=True).merge(
            another_voxelwise_df[[another_voxelwise_df.columns[i]]], left_index=True, right_index=True)
        
        # Rename the columns
        temp_df.columns = ['outcome', 'dataframe_1_voxel_i', 'dataframe_2_voxel_i']
        
        # Include interaction term if specified
        if include_interaction:
            temp_df['voxel_i_interaction'] = temp_df['dataframe_1_voxel_i'] * temp_df['dataframe_2_voxel_i']

        # Construct the regression formula based on the model type and inclusion of interaction term
        formula = 'outcome ~ dataframe_1_voxel_i + dataframe_2_voxel_i'
        if include_interaction:
            formula += ' + voxel_i_interaction'
                
        # Fit the model
        if model_type == 'linear':
            model = smf.ols(formula, data=temp_df).fit()
        elif model_type == 'logistic':
            model = smf.logit(formula, data=temp_df).fit(disp=0)
        else:
            raise ValueError(f"Unsupported model_type: {model_type}")

        # Store the results for the current voxel
        voxel_results = {
            'voxel_index': i,
            'p_values': model.pvalues.tolist(),
            'coeff_values': model.params.tolist(),
            't_values': model.tvalues.tolist(),
            'f_statistic': model.fvalue,
            'f_p_value': model.f_pvalue
        }
        
        # Store R-squared value for linear regression
        if model_type == 'linear':
            voxel_results['r_squared'] = model.rsquared

        # Append the voxel_results dictionary to the results list
        results.append(voxel_results)

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
results_df = voxelwise_multiple_neuroimages_regression(outcomes_df, 
                                                       neuroimaging_df_1, 
                                                       neuroimaging_df_2, 
                                                       model_type='linear', 
                                                       include_interaction=True)
results_df.head(1)

In [ ]:
#Unmask Dataframes
from calvin_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]

mask_data[brain_indices] = results_df['f_statistic']
    
# results_df = unmask_matrix(results_df, unmask_by='column')

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
view_and_save_nifti(mask_data, out_dir=out_dir)

Voxelwise F-statistic Comparison of Model with Interaction Versus Without Interactions
- this is for interaction of an arbritary number of neuroimaging datasets and clinical data.
- Note - the F-statistic is not a pivotal statistic under heteroscedastic conditions, thus it is not ideal for permutation analysis unless the groups sizes are equivalent. 
    -  Given this analysis performs comparison of reduced versus full models between a single group, the f-statistic is acceptable. 

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import chi2, t, f


def calculate_g_statistic(full_model, reduced_model):
    #### WORK IN PROGRESS #####
    """
    Calculates the G-statistic for a given model.
    Please note, a G-statistic is comparable to various other statistics under various conditions, 
    but it is not meant to derive p-values analytically. It is meant to derive p-values using permutation testing. 

    Parameters:
        full_model: A fitted model object from the full model.
        reduced_model: A fitted model object from the reduced model.
        heteroscedastic: Boolean. If True, the errors are assumed to be heteroscedastic, 
                         and a chi-square distribution is used to calculate the p-value. 
                         If False, the errors are assumed to be homoscedastic and a Student's 
                         t-distribution is used to calculate the p-value.

    Returns:
        G_statistic: The calculated G-statistic.
        p_value: The p-value associated with the G-statistic.
    """
    # ψ^: The estimated parameters from the full model, minus those from the reduced model
    psi_hat = full_model.params - reduced_model.params
    # C: The contrast matrix. This depends on your specific hypotheses and model.
    # Assuming that full_model and reduced_model are statsmodels regression result objects
    # Get the parameter names from both models
    full_model_params = full_model.params.index
    reduced_model_params = reduced_model.params.index

    # Initialize the contrast matrix as a zero matrix with the length of the full model params
    C = np.zeros(len(full_model_params))

    # For each parameter in the full model, if it is not in the reduced model, set the corresponding
    # element in the contrast matrix to 1
    for i, param in enumerate(full_model_params):
        if param not in reduced_model_params:
            C[i] = 1

    # Ensure that C remains a 2D array (i.e., a matrix), which is expected for matrix operations
    C = C.reshape(-1, len(C))

    # M: The design matrix from the full model
    M = full_model.model.exog

    # W: Diagonal weighting matrix.
    # Compute the residuals from your full model
    residuals = full_model.resid.to_numpy()

    # Compute the variance of residuals
    variances = np.reshape(np.var(residuals), -1)

    # Assume gn contains the variance group assignments for each observation
    # Assume R is the residual forming matrix
    # Assume epsilon_hat contains the vector of residuals

    # Initialize W as a zero matrix with the same shape as R
    W = np.zeros_like(residuals)

    # Iterate over each observation
    for n in range(len(gn)):
        # Get the variance group assignment for the n-th observation
        variance_group = gn[n]

        # Find the indices of observations belonging to the same variance group
        group_indices = np.where(gn == variance_group)[0]

        # Compute the sum of diagonal elements of R for the variance group
        sum_R = np.sum(R[group_indices, group_indices])

        # Compute the product of epsilon_hat for the variance group
        product_epsilon_hat = np.prod(epsilon_hat[group_indices])

        # Compute the diagonal element of W for the n-th observation
        W_nn = sum_R / product_epsilon_hat

        # Set the diagonal element of W for the n-th observation
        W[n, n] = W_nn


    # Calculate Λ and the inverse of it.
    Lambda_inv = np.linalg.inv(C @ M.T @ W @ M @ C.T)

    # Calculate the G-statistic
    G_statistic = psi_hat.T @ C.T @ Lambda_inv @ C @ psi_hat

    # Degrees of freedom is the rank of C
    df = np.linalg.matrix_rank(C)

    # Compute 1-tailed p-value to assess if full model is significant better than reduced model
    # Assess heteroscedasticity with the Breusch-Pagan test, p-value <0.05 indicates the linear model is heteroscedastic
    _, p_value, _, _ = het_breuschpagan(residuals, full_model.model.exog)
    print('6')
    if p_value < 0.05:
        if df == 1:
            p_value = np.NaN
            #This is equivalent to Welch's v^2, which does not have an analytical distribution
        else:
            p_value = np.NaN
            #This is equivalent to Aspen-Welch v, which does not have an analytical distribution
        print('7')
    else:
        print('8')
        if df == 1:
            print('9')
            p_value = 2 * (1 - t.cdf(np.sqrt(G_statistic), df))
            #This is equivalent to student's T, which does have an analytical distribution
        else:
            print('10')
            p_value = 1 - f.cdf(G_statistic, df, full_model.df_resid - df)
            #This is equivalent to F-ratio, which does have an analytical distribution

    return G_statistic, p_value



def handle_nan_p_values(p_value_series):
    """
    Function to handle NANs in p-values by backward filling.
    
    Parameters:
        p_value_series (pd.Series): Series containing the p-values.
    
    Returns:
        p_value_series (pd.Series): Series with NANs handled.
    """
    
    if p_value_series.isna().any():
        p_value_series.fillna(method='bfill', inplace=True)
        print('WARNING: p-values containing NAN values')
        if p_value_series.isna().any():
            p_value_series.fillna(1.0, inplace=True)
    
    return p_value_series


def fdr_correct_p_values_and_threshold_r_squared(results_df, alpha=0.05):
    """
    Performs FDR correction on the p-values in the results DataFrame and thresholds R-squared values based on the corrected p-values.
    
    Parameters:
        results_df (pd.DataFrame): DataFrame containing the p-values and R-squared values.
        alpha (float): Significance level for FDR correction.
        
    Returns:
        results_df (pd.DataFrame): DataFrame with FDR corrected p-values and thresholded R-squared values.
    """
    
    # Correct for multiple comparisons using FDR
    p_values = results_df['one_way_p_value'].values
    _, pvals_fdr, _, _ = multipletests(p_values, alpha=alpha, method='fdr_bh')
    
    # Add FDR corrected p-values to the DataFrame
    results_df['fdr_corrected_p_value'] = pvals_fdr
    
    # Threshold R-squared values based on FDR corrected p-values
    try:
        results_df['adj_r_squared_thresholded'] = results_df['adj_r_squared']
        results_df.loc[results_df['fdr_corrected_p_value'] > alpha, 'adj_r_squared_thresholded'] = 0.0
    except:
        print(' results_df["adj_r_squared_thresholded"]  not found')
    
    return results_df

def voxelwise_interaction_f_stat(outcome_df, predictor_neuroimaging_dfs, predictor_clinical_dfs, model_type='linear', manual_f_stat=False, manual_g_stat=True):
    """
    Perform voxelwise regression with interactions between the corresponding voxels from
    neuroimaging dataframes and clinical dataframes on a patient's outcome and use F-test to compare models with and without interactions.
    The F-test is the proportion of mean squared errors. When comparing two different models, F-statistic is the proprtion of the change in
    the mean squared error between the two models compared to the full model. 
    However, this means that if the second model is much worse than the first model (Larger MSE), the F-statistic can be negative. Thus, 
    negative F-statistic is considered to be equivalent to and F-statistic of zero. 
    
    degrees_of_freedom is equivalent to sample size. 
    F-statistic = ((sum_squared_residuals_1 - sum_squared_residuals_2)/(df_residuals_1 - df_residuals_2)) / (sum_squared_residuals_2/(df_residuals_2)
    MSE = sum_squared_residuals/degrees_of_freedom 
    F-statistic = delta_MSE/full_model_MSE
    
    Parameters:
        outcome_df (pd.DataFrame): DataFrame containing the outcome variable in 'outcome' column with patients in rows and 'subject_id' as index.
        predictor_neuroimaging_dfs (list of pd.DataFrame): List of DataFrames containing voxelwise neuroimaging data with patients in rows and 'subject_id' as index.
        predictor_clinical_dfs (list of pd.DataFrame): List of DataFrames containing clinical data with patients in rows and 'subject_id' as index.
        model_type (str): Specifies the type of regression model to use ('linear' or 'logistic').
        manual_f_stat (bool): If True, use the manual calculation for F-statistic and p-value. Otherwise, use anova_lm function.
    
    Returns:
        results_df (pd.DataFrame): DataFrame containing F-statistics and p-values for each voxel.
        
    Cite: 
    chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://sites.duke.edu/bossbackup/files/2013/02/FTestTutorial.pdf
    """
    
    # Number of voxels in the first neuroimaging dataframe
    n_voxels = predictor_neuroimaging_dfs[0].shape[1]

    # Initialize a list to store the results for each voxel
    results = []

    # Define lambda functions for standardization
    standardize_within_patient = lambda df: df.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
    standardize_across_patients = lambda df: df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    # Standardize outcome data across patients
    outcome_df = standardize_across_patients(outcome_df)
    if predictor_neuroimaging_dfs[0] is not None:
        # Standardize neuroimaging data within each patient
        predictor_neuroimaging_dfs = [standardize_within_patient(df) for df in predictor_neuroimaging_dfs]
    if predictor_clinical_dfs[0] is not None:
        # Standardize clinical data across patients
        predictor_clinical_dfs = [standardize_across_patients(df) for df in predictor_clinical_dfs]

    # Loop through each voxel and perform regression
    for i in tqdm(range(0, n_voxels)):
        # Create temporary dataframe with outcome and corresponding voxel from all neuroimaging dataframes and clinical data
        temp_df = outcome_df[['outcome']].copy()
        variable_names = []
        
        if predictor_neuroimaging_dfs[0] is not None:
            for j, neuroimaging_df in enumerate(predictor_neuroimaging_dfs):
                temp_df[f'dataframe_{j}_voxel_i'] = neuroimaging_df.iloc[:, i]
                variable_names.append(f'dataframe_{j}_voxel_i')
        
        if predictor_clinical_dfs[0] is not None:
            for j, clinical_df in enumerate(predictor_clinical_dfs):
                temp_df = temp_df.merge(clinical_df, left_index=True, right_index=True)
                variable_names.extend(clinical_df.columns.tolist())
            
        # Construct the regression formulas dynamically
        variables_combined = " + ".join(variable_names)
        interaction_terms = " + ".join([f"{var1}:{var2}" for idx, var1 in enumerate(variable_names) for var2 in variable_names[idx+1:]])
        
        formula_no_interaction = f'outcome ~ {variables_combined}'
        formula_interaction = f'outcome ~ {variables_combined} + {interaction_terms}'
                
        # Fit the models
        if model_type == 'linear':
            model_no_interaction = smf.ols(formula_no_interaction, data=temp_df).fit()
            model_interaction = smf.ols(formula_interaction, data=temp_df).fit()
        else:
            raise ValueError(f"Unsupported model_type: {model_type}")

        if manual_f_stat:
            # Calculate the F-statistic manually
            sum_square_1 = model_no_interaction.ssr
            sum_square_2 = model_interaction.ssr
            degrees_freedom_1 = model_no_interaction.df_resid
            degrees_freedom_2 = model_interaction.df_resid

            F_statistic = ((sum_square_1 - sum_square_2) / (degrees_freedom_1 - degrees_freedom_2)) / (sum_square_2 / degrees_freedom_2)
            if F_statistic < 0:
                F_statistic = 0 
            # Calculate the p-value using the cumulative distribution function of the F-distribution
            if np.isinf(F_statistic):
                print("Warning: Infinite F-statistic detected. Setting p-value to zero.")
                P_value = 0
            else:
                P_value = 1 - f.cdf(F_statistic, degrees_freedom_1 - degrees_freedom_2, degrees_freedom_2)
            statistic = 'manual_f_statistic'
        elif manual_g_stat:
            # This assesses heteroscedasticity at every voxel and decides what distribution the G-statistic should use
            calculate_g_statistic(model_interaction, model_no_interaction)
            statistic = 'manual_g_statistic'
        else:
            # Calculate the F-statistic and p-value using anova_lm function
            table = anova_lm(model_no_interaction, model_interaction)
            F_statistic = table['F'][1]
            if np.isinf(F_statistic):
                print("Warning: Infinite F-statistic detected. Setting p-value to zero.")
                P_value = 0.0
            else:
                P_value = table['Pr(>F)'][1]
            statistic = 'statsmodels_f_statistic'
        # Store the results for the current voxel
        # voxel_results = {
        #     'statistic_value': F_statistic,
        # }
        voxel_results = {
            'voxel_index': i,
            'statistic_value': F_statistic,
            'statistic_type': statistic,
            'one_way_p_value': P_value,
            'unc_r_squared': model_interaction.rsquared,
            'adj_r_squared': model_interaction.rsquared_adj
        }
        # Append the voxel_results dictionary to the results list
        results.append(voxel_results)

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    print('\n Example formula without interaction: ', formula_no_interaction)
    print('Example formula with interaction: ', formula_interaction)
    return results_df, temp_df

In [ ]:
outcome_df = outcomes_df
neuroimaging_dfs = [neuroimaging_df_1]  # List of neuroimaging DataFrames
clinical_dfs = [clinical_df_1]  # List of clinical DataFrames
# Note that this will only perform up to 2 way interactions. 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------
# Function calls
# Perform voxelwise regression and F-test
results_df, temp_df = voxelwise_interaction_f_stat(outcome_df, 
                                                   neuroimaging_dfs, 
                                                   clinical_dfs,
                                                   manual_f_stat=False, 
                                                   manual_g_stat=False)

# Handle NaN in p-values
results_df['one_way_p_value'] = handle_nan_p_values(results_df['one_way_p_value'])

# FDR correction and thresholding R-squared values
results_df = fdr_correct_p_values_and_threshold_r_squared(
    results_df=results_df,
)

# Output the results DataFrame
results_df.head(2)

In [ ]:
results_df['statistic_value'].to_csv(os.path.join(out_dir, 'observed_f_stats.csv'))

In [ ]:
print('Max statistic: ', results_df['statistic_value'].max())
print('Min uncorrected p-value: ', results_df['one_way_p_value'].min())
print('Min corrected p-value: ', results_df['fdr_corrected_p_value'].min())
print('Max uncorrected r-squared: ', results_df['unc_r_squared'].max())
print('Max adjusted r-squared: ', results_df['adj_r_squared'].max())

Save the Resulting Data as niftis

In [ ]:
# mask_path = None #'/Users/cu135/Dropbox (Partners HealthCare)/resources/mni_spaces/6th_gen/mni_152_gm_mask_resampled.nii'
# masking_df = None
#----------------------------------------------------------------DO NOT TOUCH
#Prepare Indices to Unmask
from calvin_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
from calvin_utils.generate_nifti import view_and_save_nifti
from nilearn import image

# unmask_matrix
# from calvin_utils.matrix_utilities import unmask_matrix
# results_df = unmask_matrix(masking_df, 
#                            mask_path=None, 
#                            mask_threshold=0.2, 
#                            unmask_by='columns', 
#                            dataframe_to_unmask_by=masking_df)
# results_df

#Perform Unmasking
if mask_path is not None:
    mni_mask = image.load_img(mask_path).get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0.2)[0]
elif masking_df is not None:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    mask = masking_df.transpose().reset_index(drop=True).copy()
    mask['mask_index'] = mask.sum(axis=1)
    brain_indices = np.where(mask['mask_index'] != 0)[0]
else:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0)[0]

# Create a boolean mask for brain_indices
bool_mask = np.zeros_like(mni_mask, dtype=bool)
bool_mask[brain_indices] = True

for statistic in results_df.columns:
    try:
        print(statistic)
        
        # Initialize the output mask with NaN values
        output_mask = np.full_like(mni_mask, np.nan)
        
        # Reinstate the values at brain_indices
        output_mask[bool_mask] = results_df[statistic]
        
        # View and save
        view_and_save_nifti(output_mask, out_dir=out_dir, output_name=statistic)
    except:
        print(f"Couldn't convert {statistic} to nifti")

Voxelwise Regression Using 1 Neuroimaging File and 1 Constant

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm
from scipy import stats
from sklearn.linear_model import Ridge


def voxelwise_regression(data_df, outcome_col, clinical_vars, voxel_start_index, model_type='linear', interaction_effects=None):
    # Replace column names with their index
    data_df.columns = list(range(len(data_df.columns)))
    
    n_voxels = len(data_df.columns[voxel_start_index:])
    print(n_voxels)
    
    p_values = [[] for _ in range(n_voxels)]
    coeff_values = [[] for _ in range(n_voxels)]
    t_values = [[] for _ in range(n_voxels)]

    indep_vars_str = " + ".join(f"var{x}" for x in clinical_vars)

    if interaction_effects:
        interaction_terms_str = " + ".join([f"var{data_df.columns[a]}_interact_connectivity" for a in interaction_effects])
        indep_vars_str = f"{indep_vars_str} + connectivity + {interaction_terms_str}"

    if model_type == 'linear':
        formula = f'outcome ~ {indep_vars_str}'
        print("Regression formula (linear):", formula)
    elif model_type == 'logistic':
        formula = f'outcome ~ {indep_vars_str}'
        print("Regression formula (logistic):", formula)
    else:
        raise ValueError(f"Unsupported model_type: {model_type}")

    for i in tqdm(range(0, n_voxels)):
        temp_df = data_df.iloc[:, [outcome_col] + clinical_vars + [voxel_start_index + i]]
        temp_df.columns = ['outcome'] + [f"var{x}" for x in clinical_vars] + ['connectivity']

        # Add interaction columns to the temp_df
        if interaction_effects:
            for idx in interaction_effects:
                interaction_col_name = f'var{data_df.columns[idx]}_interact_connectivity'
                temp_df[interaction_col_name] = temp_df[f"var{data_df.columns[idx]}"] * temp_df['connectivity']

        if model_type == 'linear':
            model = smf.ols(formula, data=temp_df).fit()
        elif model_type == 'logistic':
            temp_df['outcome'] = (temp_df['outcome'] > 0.5).astype(int)
            model = smf.logit(formula, data=temp_df).fit(disp=0)
        elif model_type == 'ridge':
            ridge = Ridge(alpha=1.0)
            X = sm.add_constant(temp_df.drop(columns='outcome'))
            y = temp_df['outcome']
            ridge.fit(X, y)
        else:
            raise ValueError(f"Unsupported model_type: {model_type}")

        if model_type =='ridge':
            # Perform p and t value extraction for the ridge regressions
            n = len(temp_df)
            k = len(temp_df.columns)
            dof = n - k
            y_pred = ridge.predict(X)
            residuals = y - y_pred
            mse = np.sum(residuals**2) / dof
            se = np.sqrt(np.diagonal(mse * np.linalg.inv(np.dot(X.T, X))))
            t_values_ridge = ridge.coef_ / se
            p_values_ridge = [2 * (1 - stats.t.cdf(abs(t_value), dof)) for t_value in t_values_ridge]

            for j, (p_value, coef, t_value) in enumerate(zip(p_values_ridge, ridge.coef_, t_values_ridge)):
                p_values[i].append(p_value)
                coeff_values[i].append(coef)
                t_values[i].append(t_value)
        else:
            for j, (p_value, coef, t_value) in enumerate(zip(model.pvalues, model.params, model.tvalues)):
                p_values[i].append(p_value)
                coeff_values[i].append(coef)
                t_values[i].append(t_value)

    columns = ['Intercept'] + [f"var{x}" for x in clinical_vars] + ['connectivity'] + [f'var{var}_interact_connectivity' for var in interaction_effects]
    p_values_df = pd.DataFrame(p_values, columns=columns)
    coeff_values_df = pd.DataFrame(coeff_values, columns=columns)
    t_values_df = pd.DataFrame(t_values, columns=columns)

    return p_values_df, coeff_values_df, t_values_df, model



import numpy as np
from nilearn.glm import fdr_threshold
from calvin_utils.z_score_matrix import z_score_matrix
from scipy.stats import zscore

In [ ]:
#Check input for NANs and Constant values
regression_df.isnull().values.any()
test_df = regression_df.loc[:, regression_df.apply(pd.Series.nunique) != 1]
if test_df.shape != regression_df.shape:
    print('Note, there are constant columns in the regression dataframe. This may cause unstable regression results')

In [ ]:
#Matrix input should be oriented such that voxels and other clinical information are in columns, with patients in rows
p_values_df, coeff_values_df, t_values_df, model = voxelwise_regression(data_df=regression_df, outcome_col=0, clinical_vars=[1], voxel_start_index=2, model_type='linear', interaction_effects=[1])

In [ ]:
print("Shape of p_values_df:", p_values_df.shape)
print("Shape of coeff_values_df:", coeff_values_df.shape)
print("Shape of t_values_df:", t_values_df.shape)
print("Shape of regression_df:", regression_df.shape)
print(model.summary2())

In [ ]:
import numpy as np
from nilearn.glm import fdr_threshold
from calvin_utils.z_score_matrix import z_score_matrix
from scipy.stats import zscore
from statsmodels.stats import multitest
from statsmodels.stats.multitest import fdrcorrection


def insert_outcomes_to_matrix(p_values_df, coeff_values_df, t_values_df, brain_indices, mask_data, view_and_save_nifti, out_dir, adjustment_method='nilearn', strict=False):
    '''
    This function takes the outcomes of the voxelwise regression and generates NIfTI images from them.
    It also calculates FDR-adjusted p-values and masks the coefficients based on the adjusted p-values.
    
    Parameters:
    p_values_df (pd.DataFrame): DataFrame containing p-values.
    coeff_values_df (pd.DataFrame): DataFrame containing coefficients.
    t_values_df (pd.DataFrame): DataFrame containing t-values.
    brain_indices (array): Indices of the brain voxels.
    mask_data (array): Mask data for the brain.
    view_and_save_nifti (function): Function to view and save NIfTI images.
    out_dir (str): Output directory for saving the NIfTI images.
    adjustment_method (str, optional): Method to use for FDR adjustment. Defaults to 'nilearn'.
                                      Accepted values are 'nilearn' and 'statsmodels'.
    
    Raises:
    ValueError: If adjustment_method is not 'nilearn' or 'statsmodels'.
    '''
    
    # Convert DataFrames to NumPy arrays
    p_values = p_values_df.to_numpy()
    coeff_values = coeff_values_df.to_numpy()
    t_values = t_values_df.to_numpy()

    # Iterate over statistical values
    for stat_name, stat_values in zip(["p_values", "coeff_values", "t_values"], [p_values, coeff_values, t_values]):
        for col_idx, col_name in enumerate(p_values_df.columns):
            matrix = mask_data.copy()
            matrix[brain_indices] = stat_values[:, col_idx]

            # Generate NIfTI image for the current statistical value and term
            out_file = f"{out_dir}/{col_name}/{stat_name}.nii"
            view_and_save_nifti(matrix=matrix, out_dir=out_file)

            if stat_name == 't_values':
                if adjustment_method == 'nilearn':
                    # Calculate FDR-adjusted p-values based on t-values
                    z_scored_t_values = zscore(np.nan_to_num(t_values[:, col_idx]))
                    z_scored_t_values = np.nan_to_num(z_scored_t_values, nan=0, posinf=0, neginf=0)
                    print(np.max(z_scored_t_values), np.min(z_scored_t_values))
                    threshold = fdr_threshold(z_scored_t_values, 0.05)
                    print('FDR Threshold at : ', threshold)
                    
                    # Mask the coefficients based on FDR-adjusted z_scored_t_values
                    fdr_masked_coeff = np.where(z_scored_t_values < threshold, coeff_values[:, col_idx], 0)
                                        
                elif adjustment_method == 'statsmodels':
                    # Calculate FDR-adjusted p-values using statsmodels
                    if strict:
                        method = 'poscor'
                    else:
                        method = 'indep'
                    _, fdr_adj_p_values = fdrcorrection(p_values[:, col_idx], method='indep')
                    print('unadjusted p-values min and max at : ', np.min(p_values[:, col_idx]), np.max(p_values[:, col_idx]))
                
                    print('FDR adjusted p-values min and max at : ', np.min(fdr_adj_p_values), np.max(fdr_adj_p_values))
                    # Mask the coefficients based on FDR-adjusted p-values
                    fdr_masked_coeff = np.where(fdr_adj_p_values < 0.05, coeff_values[:, col_idx], 0)
                
                else:
                    raise ValueError(f"adjustment_method model_type: {adjustment_method}, nilearn or 'statsmodels only currently supported")
                
                # Generate NIfTI image for FDR-masked coefficients
                matrix = mask_data.copy()
                matrix[brain_indices] = fdr_masked_coeff
                out_file = f"{out_dir}/{col_name}/fdr_masked_coeff_{stat_name}.nii"
                view_and_save_nifti(matrix=matrix, out_dir=out_file)
                
                # Generate NIfTI image for FDR-madjusted p-values
                new_matrix = mask_data.copy()
                new_matrix[brain_indices] = fdr_adj_p_values
                out_file = f"{out_dir}/{col_name}/fdr_adjusted_p_values.nii"
                view_and_save_nifti(matrix=new_matrix, out_dir=out_file)

In [ ]:
# Assuming p_values_df, coeff_values_df, and t_values_df are obtained from voxelwise_regression function
# Assuming brain_indices and mask_data are predefined
# Assuming view_and_save_nifti function is predefined
from calvin_utils.generate_nifti import view_and_save_nifti
out_dir = out_dir

#only operate within the brain mask
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]

insert_outcomes_to_matrix(p_values_df, coeff_values_df, t_values_df, brain_indices, mask_data, view_and_save_nifti, out_dir, adjustment_method='statsmodels', strict=True)


# Mask Coefficient by PALM

In [ ]:
# coeff_values_df.iloc[:, -1] = np.where(palm_p_values.iloc[:, -1] < 0.05, coeff_values_df.iloc[:, -1], 0)
mask_data[brain_indices]

In [ ]:
# palm_p_values = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/memory/final_analyses/interaction_analysis/whole_brain_analysis/permutation_results/10185_summed_coefficient_pp_values (2).csv')
# coeff_values_df.iloc[:, -1] = np.where(palm_p_values.iloc[:, -1] < 0.05, coeff_values_df.iloc[:, -1], 0)
mask_data[brain_indices] = coeff_values_df.iloc[:, -1]
#Generate a nifti for coefficients thresholded to uncorrected p-values
view_and_save_nifti(mask_data, (out_dir+'/coefficient_thresholded_to_palm_p_values'))

## Perform Permutation Testing

This is generally best done with submission to a cluster.

In [ ]:
from calvin_utils.multiprocessing import permute_brain
#Permute the data, returning a matrix organized such that shape = [voxels, patient, permutation]
#Convert dataframe to numpy for time savings
#Is actually receiving the float values, not the indices. 
permuted_array = permute_brain(prepped_matrix.to_numpy(), n_permutations=1)

#---Visualize a given permutation
# retrive brain mask
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]

#Retrieve patient data
test_matrix = prepped_matrix.iloc[:,0].to_numpy()

#reindex patient data by permutation
permutation = permuted_array[:,0,0]
test_matrix[brain_indices] = permutation
view_and_save_nifti(test_matrix, (out_dir+'/verification'))

In [ ]:
import numpy as np
import statsmodels.formula.api as smf

def calculate_empirical_p(observed_t_value, sim_t_values):
    """
    Calculate empirical p-value from a distribution of empiric t-values and an observed t-value.
    """
    # count the number of simulated t-values that are as extreme or more extreme than the critical t-value
    extreme_count = np.sum(np.abs(sim_t_values) >= observed_t_value)

    # divide the number of extreme values by the total number of simulated values to get the empirical p-value
    empirical_p = extreme_count / len(sim_t_values)

    return empirical_p

def process_voxel(i):
    #Do not calculate on zero values as intercept will be applied in the regressoin
    if np.sum(prepped_matrix.iloc[i,:]) != 0:
        #Assign a temporary dataframe with values that the statsmodels model is expecting
        ##the plan is to use patient ID to intersect the voxelwise data to the associated clinical data
        temp_df = data_df.copy()
        #Names of the columns are as per data_df. The voxel coumn's name is the integer of the voxel
        temp_df = temp_df.merge(prepped_matrix.iloc[i,:].transpose(), left_index=True, right_index=True)
        #Rename the column of the voxel to 'connectivity' to prepare it for the statsmodels input requirements
        temp_df = temp_df.rename(columns={i: "connectivity"}, errors="raise")
        
        #Fit model. Outcomes are always first
        results = smf.ols(f'{temp_df.columns[0]} ~ {temp_df.columns[1]}*{temp_df.columns[2]}', data=temp_df).fit()
        #Extract the t statistic of relevance from the model at this voxel. Interaction effect is len(indep_vars)+1 (intercept, main1, main2, interaction)
        voxel_t_value = results.tvalues[3]
        
        #Assess the t value against the permutation's distribution 
        t_value_list = []       
        for j in range(permuted_array.shape[2]):
            # Assign a temporary dataframe with values that the statsmodels model is expecting
            temp_df_permuted = data_df.copy()
            #Permute the associated data
            temp_df_permuted[temp_df.columns[1]] = np.random.permutation(temp_df[temp_df.columns[1]])
            #Retrive the already permuted voxel for all patients at this permutation
            temp_df_permuted[temp_df.columns[2]] = permuted_array[i,:,j]
            
            # Fit model. Outcomes are always first
            results_permuted = smf.ols(f'{temp_df_permuted.columns[0]} ~ {temp_df_permuted.columns[1]}*{temp_df_permuted.columns[2]}', data=temp_df_permuted).fit()
            # Extract the t statistic of relevance from the model at this voxel. Interaction effect is len(indep_vars)+1 (intercept, main1, main2, interaction)
            voxel_t_value_permuted = results_permuted.tvalues[3]
            t_value_list.append(voxel_t_value_permuted)
            
        #Calculate the p-value from the observed t-value copmared to the empiric permuted distribution of t-values
        voxelwise_p_value = calculate_empirical_p(voxel_t_value_permuted, t_value_list)
    else:
        #If voxel is zero-connectivity, assign zero so as to avoid application of intercept
        voxelwise_p_value = 0
    return voxelwise_p_value


In [ ]:
import numpy as np
import statsmodels.formula.api as smf

def calculate_empirical_p(observed_t_value, sim_t_values):
    """
    Calculate empirical p-value from a distribution of empiric t-values and an observed t-value.
    """
    # count the number of simulated t-values that are as extreme or more extreme than the critical t-value
    extreme_count = np.sum(np.abs(sim_t_values) >= observed_t_value)

    # divide the number of extreme values by the total number of simulated values to get the empirical p-value
    empirical_p = extreme_count / len(sim_t_values)

    return empirical_p

def permutation_testing(matrix_to_test, t_matrix):
    '''
    The plan will be to create one dataframe which can be used to evaluate. 
    Permuted age will go in the first column
    Connectivity will be transposed and attached to the second column
    the 'connectivity' column name will move for every voxel and the t value will be calculated
    
    args: 
    matrix_to_test is the isolated prepared dataframe to utilize.
    t_matrix is the pre-calculated matrix of t values from the non-permuted brains
    
    performs:
    Will calculate t-value for each voxel. Then, it will assess if the t-value is less than the t-value from the non-permuted brain
    If the permuted t-value is less than the t-value from the non-permuted brain, then that voxels is set to 0. 
    
    returns:
    simply a binary matrix of 1s and 0s
    '''
    print('in function')
    #Permute outcomes
    permuted_outcomes = np.apply_along_axis(np.random.permutation, 0, matrix_to_test[:,0])
    
    #Permute the age
    permuted_age = np.apply_along_axis(np.random.permutation, 0, matrix_to_test[:,1])    
    
    #Permute the Voxels across and within patients (PALM default)
    permuted_across = np.apply_along_axis(np.random.permutation, 0, matrix_to_test[:,2:])
    permuted_within = np.apply_along_axis(np.random.permutation, 1, permuted_across)
    print('permutations complete')
    #Reconstruct a dataframe
    permuted_dataframe = pd.DataFrame(np.concatenate((permuted_outcomes.reshape(-1, 1), permuted_age.reshape(-1, 1), permuted_within), axis=1))
    print('dataframe completed')
    #Perform calculation of t values for each voxel
    permuted_t_values = []
    for i in tqdm(range(len(t_matrix)), desc='Subprocess Progress'):
        print('on loop ', i)
        #The dataframe has the structure: outcome, age, voxels
        #Fit the model. Voxel columns must be strings. 
        results_permuted = smf.ols(f'{permuted_dataframe.columns[0]} ~ {permuted_dataframe.columns[1]}*{permuted_dataframe.columns[i+2]}', data=permuted_dataframe).fit()
        permuted_t_values.append(results_permuted.tvalues[3])
    
    #Binarize permuted t-values by comparison to non-permuted t-values
    #Any t-values that are higher are not significant, so that are = 1.
    return np.where(permuted_t_values>t_matrix, 1, 0)

In [ ]:
##Prepare data for analysis

# Retrieve brain mask
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]
t_matrix = t_df.to_numpy()[brain_indices, -1]

# Isolate only the values of interest from the associated data
matrix_to_test = prepped_matrix.iloc[brain_indices, :]

if 'response_df' in locals():
    t_df = response_df['t_values_topology'].to_numpy()[brain_indices]
else:
    print('not found in locals, must import')
    t_df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/response_topology/voxelwise_glm/age_interaction_rios_vtas/t_values_topology/t_values.csv', index_col=None)
#index t matrix at last column to make sure no index included 
t_matrix = t_df.to_numpy()[brain_indices, -1]

#Prepare the data for assessment
matrix_to_test = data_df.merge(matrix_to_test.transpose(), left_index=True, right_index=True)
#Reduce size of data to ease computations
matrix_to_test = matrix_to_test.astype(np.float16)

n_permutations = 100
display(matrix_to_test)
# matrix_to_test.to_csv(out_dir+'/t_values_topology/matrix_to_test.csv')

In [ ]:
print(len(t_matrix))
print(np.shape(matrix_to_test))
# for i in range(len(t_matrix)):
#     print(matrix_to_test.columns[i+2])

In [ ]:
#Perform analysis
from tqdm import tqdm
import concurrent
from calvin_utils.multiprocessing import whole_brain_permutation_test
#----------------------------------------------------------------Do not Touch Below
# Multiprocess Results
p_matrix = np.ones_like(t_matrix)

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    #Begin submitting the masked data to the permutor
    results = []
    for i in tqdm(range(n_permutations), desc="Jobs Launched"):
        #Assign the permuted data to a worker. return the result
        result = executor.submit(whole_brain_permutation_test, matrix_to_test, t_matrix)
        results.append(result)
        
        # Limit number of workers at given time to prevent memory pressure issues
        # jobs[result] = i #Add job to dict of ongoing jobs
        # if len(jobs) > max_jobs-1: #Check number of ongoing jobs
        #     completed_jobs, _ = concurrent.futures.wait(jobs.keys(), return_when=concurrent.futures.FIRST_COMPLETED)
        #     # Remove the completed job from the dict of ongoing jobs
        #     del jobs[completed_jobs.pop()]

    progress_bar = tqdm(total=n_permutations, desc="Jobs Finalized")
    for i, result in enumerate(concurrent.futures.as_completed(results)):
        #Input the permuted data into the array
        extracted_p_values = result.result()
        p_matrix = p_matrix + extracted_p_values
        
        #Update visualization
        progress_bar.update()
    progress_bar.close()

In [ ]:
p_df = pd.DataFrame(p_matrix)
p_df.to_csv(out_dir+'/t_values_topology/p_vals_2.csv',index=False, header=False)

## Visualize PALM Results

This allows recomposition of the PALM results from the cluster

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results'
#data path
p_values_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/10000_summed_f_statp_values.csv'
t_values_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/observed_f_stats.csv'
num_perms = 5

#import data
if 'response_df' in locals():
    t_df = response_df['t_values_topology'].to_numpy()[brain_indices]
else:
    print('not found in locals, must import')
    t_df = pd.read_csv(t_values_dir, index_col=None)
p_df = pd.read_csv(p_values_dir, header=0, index_col=None)

#Scale the p_values
print('---Information on p-Values---')
print('Nonzero p_values found: ', np.count_nonzero(p_df[p_df.iloc[:,-1] < 0.05]))
print('Lowest p_values: ', np.min(p_df.iloc[:,-1]))
display(p_df)

Handle NANs or Unclean Data

In [ ]:
t_df.dropna(inplace=True)
p_df.dropna(inplace=True)
t_df.pop('Unnamed: 0')
t_df.max()
p_df.max()
# t_df

Unmask the matrices

In [ ]:
from calvin_utils.matrix_utilities import unmask_matrix
from nilearn import masking, image
from nimlab import datasets as nimds
from nilearn import image

mask = nimds.get_img("mni_icbm152")
mask_data = mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]

#Insert 
p_unmasked = np.copy(mask_data)
p_unmasked[brain_indices] = p_df.values.flatten()
print(np.max(p_unmasked))

other_unmasked = np.copy(mask_data)
other_unmasked[brain_indices] = t_df.values.flatten()
print(np.max(other_unmasked))

Mask statistic of interest by p-values

In [ ]:
#Threshold by p_values
other_unmasked = np.where(p_unmasked < 0.05, other_unmasked, 0)
print(np.max(p_unmasked))
print(np.max(other_unmasked))

View Statistic of interest

In [ ]:
other_unmasked_img = view_and_save_nifti(other_unmasked, out_dir=(os.path.join(out_dir, f'palm_statistic_topology')))
other_unmasked_img

View Base Image

In [ ]:
base_img = view_and_save_nifti(other_unmasked, out_dir=(os.path.join(out_dir, f'unmasked_statistic_topo')))
base_img

View R2

In [ ]:
from nilearn import image
r_path = ''
r2_img = image.load_img(r_path)
r2_img_data = r2_img.get_fdata().flatten()
r2_masked = np.where(p_unmasked < 0.05, r2_img_data, 0)

masked_r2_img = view_and_save_nifti(r2_masked, out_dir=(os.path.join(out_dir, f'masked_r2')))
masked_r2_img

View Adjusted R2

In [ ]:
from nilearn import image
adjr_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/adj_r_squared_generated_nifti.nii'
adjr2_img = image.load_img(adjr_path)
adjr2_img_data = adjr2_img.get_fdata().flatten()
adjr2_masked = np.where(p_unmasked < 0.05, adjr2_img_data, 0)

masked_adjr2_img = view_and_save_nifti(adjr2_masked, out_dir=(os.path.join(out_dir, f'masked_adjr2')))
masked_adjr2_img